In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn import preprocessing

In [3]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

## Read Data from Adult Dataset and Clean

In [4]:
adult_data_path = 'raw_adult/adult.data'
adult_test_path = 'raw_adult/adult.test'
cols = ['age','workclass','fnlwgt','education','education-num','marital-status',
        'occupation','relationship','race','sex','capital-gain', 'capital-loss',
        'hours-per-week', 'native-country','compensation']
a = (pd.read_csv(adult_data_path,
                 names=cols,
                 sep=', ',
                 engine='python')
     .append(pd.read_csv(adult_test_path,
                         skiprows=1,
                         names=cols,
                         sep=', ',
                         engine='python'))
     .replace({'<=50K.' : '<=50K',
               '>50K.'  : '>50K'}))

In [5]:
a = (a[(a['workclass']!='?')&
       (a['occupation']!='?')&
       (a['workclass']!='Without-pay')]
     .reset_index(drop=True))
a['idx'] = a.index

Map the very small `Armed-Forces` category of `occupation` to `Protective-serv`.

In [6]:
a.loc[a['occupation']=='Armed-Forces','occupation'] = 'Protective-serv'

### Map **Ages**, **Education**, **Workclass**, and **Weekly-Hours** to smaller category set.

In [7]:
a['age'] = a['age']//10
a['age'] = (a['age']*10).astype(str) + 's'
a['age'].unique()

array(['30s', '50s', '20s', '40s', '10s', '70s', '60s', '90s', '80s'],
      dtype=object)

In [8]:
a.loc[a['workclass'].isin(['State-gov', 'Federal-gov', 'Local-gov']), 
      'employment-type'] = 'Government'
a.loc[a['workclass'].isin(['Self-emp-not-inc', 'Self-emp-inc']),      
      'employment-type'] = 'Self-Employed'
a.loc[a['workclass'].isin(['Private']),                               
      'employment-type'] = 'Privately-Employed'
a['employment-type'].unique()

array(['Government', 'Self-Employed', 'Privately-Employed'], dtype=object)

In [9]:
a.loc[a['education-num'] <= 8,          'education'] = 'Less than High School'
a.loc[a['education-num'].isin([ 9,10]), 'education'] = 'High School'
a.loc[a['education-num'].isin([11,12]), 'education'] = 'Associates'
a.loc[a['education-num'].isin([13]),    'education'] = 'Bachelors'
a.loc[a['education-num'].isin([14]),    'education'] = 'Masters'
a.loc[a['education-num'].isin([15,16]), 'education'] = 'PhD/Professional'
a['education'].unique()

array(['Bachelors', 'High School', 'Less than High School', 'Masters',
       'Associates', 'PhD/Professional'], dtype=object)

In [10]:
a.loc[a['hours-per-week'] < 20, 'weekly-hours'] = '<20'
a.loc[(a['hours-per-week'] < 40)&
      (a['weekly-hours'].isna()), 'weekly-hours'] = '20+'
a.loc[(a['hours-per-week'] < 60)&
      (a['weekly-hours'].isna()), 'weekly-hours'] = '40+'
a.loc[(a['hours-per-week'] < 80)&
      (a['weekly-hours'].isna()), 'weekly-hours'] = '60+'
a.loc[(a['hours-per-week'] >= 80)&
      (a['weekly-hours'].isna()), 'weekly-hours'] = '80+'
a['weekly-hours'].unique()

array(['40+', '<20', '80+', '20+', '60+'], dtype=object)

### Convert All Categories to Numeric values

In [11]:
a['age raw'] = a['age']
a['employment-type raw'] = a['employment-type']
a['education raw'] = a['education']
a['occupation raw'] = a['occupation']
a['race raw'] = a['race']
a['sex raw'] = a['sex']
a['weekly-hours raw'] = a['weekly-hours']
a['compensation raw'] = a['compensation']

In [12]:
a['age'] = a['age'].astype('category')
a['age'] = a['age'].cat.codes
a[['age','age raw','idx']].groupby(['age','age raw']).count()

,,idx
age,age raw,
0,10s,2067
1,20s,11169
2,30s,12615
3,40s,10498
4,50s,6365
5,60s,2559
6,70s,592
7,80s,100
8,90s,47


In [13]:
a['employment-type'] = a['employment-type'].astype('category')
a['employment-type'] = a['employment-type'].cat.codes
a[['employment-type','employment-type raw','idx']].groupby(['employment-type',
                                                            'employment-type raw']).count()

,,idx
employment-type,employment-type raw,
0,Government,6549
1,Privately-Employed,33906
2,Self-Employed,5557


In [14]:
a['education'] = a['education'].astype('category')
a['education'] = a['education'].cat.codes
a[['education','education raw','idx']].groupby(['education',
                                                'education raw']).count()

,,idx
education,education raw,
0,Associates,3505
1,Bachelors,7772
2,High School,24991
3,Less than High School,5768
4,Masters,2590
5,PhD/Professional,1386


In [15]:
a['occupation'] = a['occupation'].astype('category')
a['occupation'] = a['occupation'].cat.codes
a[['occupation','occupation raw','idx']].groupby(['occupation',
                                                  'occupation raw']).count()

,,idx
occupation,occupation raw,
0,Adm-clerical,5608
1,Craft-repair,6111
2,Exec-managerial,6085
3,Farming-fishing,1482
4,Handlers-cleaners,2070
5,Machine-op-inspct,3020
6,Other-service,4921
7,Priv-house-serv,242
8,Prof-specialty,6172


In [16]:
a['race'] = a['race'].astype('category')
a['race'] = a['race'].cat.codes
a[['race','race raw','idx']].groupby(['race','race raw']).count()

,,idx
race,race raw,
0,Amer-Indian-Eskimo,435
1,Asian-Pac-Islander,1422
2,Black,4355
3,Other,375
4,White,39425


In [17]:
a['sex'] = a['sex'].astype('category')
a['sex'] = a['sex'].cat.codes
a[['sex','sex raw','idx']].groupby(['sex','sex raw']).count()

,,idx
sex,sex raw,
0,Female,14912
1,Male,31100


In [18]:
a['weekly-hours'] = a['weekly-hours'].astype('category')
a['weekly-hours'] = a['weekly-hours'].cat.codes
a[['weekly-hours','weekly-hours raw','idx']].groupby(['weekly-hours',
                                                      'weekly-hours raw']).count()

,,idx
weekly-hours,weekly-hours raw,
0,20+,8200
1,40+,32032
2,60+,3253
3,80+,496
4,<20,2031


In [19]:
a['compensation'] = a['compensation'].astype('category')
a['compensation'] = a['compensation'].cat.codes
a[['compensation','compensation raw','idx']].groupby(['compensation',
                                                      'compensation raw']).count()

,,idx
compensation,compensation raw,
0,<=50K,34592
1,>50K,11420


### Subset the feature space, if required

In [20]:
a = a[['idx',
       'age',
       'employment-type',
       'education',
       'occupation',
       'race',
       'sex',
       'weekly-hours',
       'compensation']].copy()
print(a.shape)
a.head()

(46012, 9)


,idx,age,employment-type,education-num,occupation,race,sex,weekly-hours,compensation
0,0,2,0,13,0,4,1,1,0
1,1,4,2,13,2,4,1,4,0
2,2,2,1,9,4,4,1,1,0
3,3,4,1,7,4,2,1,1,0
4,4,1,1,13,8,2,0,1,0


Write to file

In [21]:
a.to_csv('clean/adult_clean.csv',
         index=False)